In [6]:
import pandas as pd
import numpy as np

In [7]:
cust = pd.read_csv('customers.csv')
pro = pd.read_csv('products.csv')
tra = pd.read_csv('transactions.csv')

In [8]:
cust.shape

(2000, 6)

In [9]:
pro.shape

(150, 5)

In [10]:
tra.shape

(10015, 7)

In [11]:
cust.dtypes

customer_id        int64
customer_name     object
email             object
signup_date       object
country           object
age              float64
dtype: object

In [12]:
pro.dtypes

product_id        int64
product_name     object
category         object
supplier_id       int64
cost_price      float64
dtype: object

In [13]:
tra.dtypes

transaction_id      int64
customer_id         int64
product_id          int64
quantity          float64
price             float64
timestamp          object
payment_method     object
dtype: object

In [14]:
cust.head(3)

,customer_id,customer_name,email,signup_date,country,age
0,1,Customer41,customer410@yahoo.com,2022-01-28,UK,30.0
1,2,Custome41,custome411@email.com,2023-10-28,Mexico,29.0
2,3,customer41,customer412@email.com,2023-04-05,Australia,69.0


In [15]:
pro.head(3)

,product_id,product_name,category,supplier_id,cost_price
0,1,Product_OFF_0,Toys,28,113.16
1,2,Product_SPO_1,Books,43,166.06
2,3,Product_SPO_2,Books,47,128.13


In [16]:
tra.head(3)

,transaction_id,customer_id,product_id,quantity,price,timestamp,payment_method
0,4292,130,89,1.0,112.87,2024-08-17 09:13:00,Credit Card
1,9107,271,98,1.0,211.90,2023-11-17 20:50:00,PayPal
2,7842,66,33,1.0,225.04,2023-03-21 23:23:00,Bank Transfer


In [17]:
tra['timestamp'] = pd.to_datetime(tra['timestamp'], errors='coerce')
print("\nEarliest Date:", tra['timestamp'].min())
print("Latest Date:", tra['timestamp'].max())


tra['hour']  = tra['timestamp'].dt.hour
tra['day']   = tra['timestamp'].dt.day_name()
tra['month'] = tra['timestamp'].dt.month_name()



Earliest Date: 2023-01-01 00:39:00
Latest Date: 2024-12-31 22:18:00


In [18]:
miss = cust.isnull().sum()
print(miss)
miss1 = pro.isnull().sum()
print(miss1)
miss2 = tra.isnull().sum()
print(miss2)

customer_id        0
customer_name      0
email             53
signup_date        0
country            0
age              100
dtype: int64
product_id      0
product_name    0
category        3
supplier_id     0
cost_price      0
dtype: int64
transaction_id      0
customer_id         0
product_id          0
quantity           97
price             102
timestamp           0
payment_method    101
hour                0
day                 0
month               0
dtype: int64


In [19]:
tra['transaction_id'].duplicated().sum()

np.int64(15)

In [20]:
for i in tra.columns:
    if(tra[i].dtypes == 'int64'):
        tra.fillna(tra[i].mean(), inplace=True)

for i in cust.columns:
    if(cust[i].dtypes == 'int64'):
        cust.fillna(cust[i].mean(), inplace=True)

for i in pro.columns:
    if(pro[i].dtypes == 'int64'):
        pro.fillna(pro[i].mean(), inplace=True)
    

In [21]:
miss = cust.isnull().sum()
print(miss)
miss1 = pro.isnull().sum()
print(miss1)
miss2 = tra.isnull().sum()
print(miss2)

customer_id      0
customer_name    0
email            0
signup_date      0
country          0
age              0
dtype: int64
product_id      0
product_name    0
category        0
supplier_id     0
cost_price      0
dtype: int64
transaction_id    0
customer_id       0
product_id        0
quantity          0
price             0
timestamp         0
payment_method    0
hour              0
day               0
month             0
dtype: int64


In [29]:
tra['revenue'] = tra['quantity'] * tra['price']
print(tra[['transaction_id', 'quantity', 'price', 'revenue']].head())
print(tra['revenue'].sum())


   transaction_id  quantity   price  revenue
0            4292       1.0  112.87   112.87
1            9107       1.0  211.90   211.90
2            7842       1.0  225.04   225.04
3            3485       1.0   72.20    72.20
4            8929       1.0  315.22   315.22
93335180.80365734


In [30]:
full_data = (
    tra
    .merge(cust, on='customer_id', how='left')
    .merge(pro,  on='product_id',  how='left')
)

In [31]:
full_data['profit_margin'] = (full_data['price'] - full_data['cost_price']) / full_data['price'] * 100
print(full_data[['price', 'cost_price', 'profit_margin']].head())

    price  cost_price  profit_margin
0  112.87       59.72      47.089572
1  211.90       84.89      59.938650
2  225.04      103.61      53.959296
3   72.20       39.53      45.249307
4  315.22      127.00      59.710678


In [37]:
revenue = full_data.groupby('customer_id')['revenue'].sum().sort_values(ascending=False).reset_index()
revenue.columns = ['customer_id', 'total_revenue']
print(revenue.head())

   customer_id  total_revenue
0          273   3.126761e+06
1          254   2.296137e+06
2          944   2.260295e+06
3          407   2.109814e+06
4         1646   2.027093e+06


In [41]:
num = full_data.groupby('customer_id')['transaction_id'].nunique().reset_index()
num.columns = ['customer_id', 'transaction_id']
print(num.head())

   customer_id  transaction_id
0            1               5
1            2               3
2            3              30
3            4               8
4            5               5


In [43]:
avg = full_data.groupby('customer_id')['revenue'].mean().reset_index()
avg.columns = ['customer_id', 'avg_revenue']
print(avg.head())

   customer_id  avg_revenue
0            1   284.998000
1            2   213.996667
2            3   855.279256
3            4   547.953750
4            5   384.196000


In [51]:
prod = full_data.groupby('product_name')['revenue'].sum().sort_values(ascending=False).reset_index()
prod.columns = ['product_name','total_revenue']
print(prod.head(10))

      product_name  total_revenue
0   Product_BEA_66   5.906845e+06
1   Product_OFF_59   4.412456e+06
2   Product_BEA_62   3.711972e+06
3   Product_OFF_90   3.235839e+06
4   Product_HOM_89   3.138393e+06
5  Product_BEA_102   3.133425e+06
6    Product_OFF_0   3.008467e+06
7   Product_BOO_14   2.600273e+06
8   Product_ELE_67   2.588676e+06
9   Product_OFF_47   2.329346e+06


In [53]:
prod = full_data.groupby('product_name')['quantity'].sum().sort_values(ascending=False).reset_index().round(0)
prod.columns = ['product_name','quantity']
print(prod.head(10))

      product_name  quantity
0   Product_BEA_66   25368.0
1   Product_HOM_89   20676.0
2   Product_BEA_62   20433.0
3   Product_ELE_67   15616.0
4  Product_BEA_145   15183.0
5   Product_FOO_25   15143.0
6    Product_OFF_0   15097.0
7   Product_SPO_40   11745.0
8   Product_HOM_27   11116.0
9   Product_OFF_59   11109.0


In [55]:
pm = full_data.groupby('product_name')['profit_margin'].mean().reset_index().sort_values(by='profit_margin', ascending=False).round(2)
pm.columns = ['product_name','avg_profit_margin']
print(pm.head(10))

        product_name  avg_profit_margin
4    Product_AUT_120              48.93
98     Product_OFF_0              48.66
115   Product_OFF_85              48.51
121  Product_SPO_141              48.35
46    Product_BOO_83              47.11
141   Product_TOY_29              47.09
67    Product_CLO_87              47.04
74    Product_ELE_74              46.92
70   Product_ELE_103              46.88
21   Product_BEA_149              46.56
